In [1]:
import numpy as np
import pandas as pd

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ods-churn-24/currency_rk.csv
/kaggle/input/ods-churn-24/report_dates.csv
/kaggle/input/ods-churn-24/mcc_codes.csv
/kaggle/input/ods-churn-24/clients.csv
/kaggle/input/ods-churn-24/train.csv
/kaggle/input/ods-churn-24/sample_submit_naive.csv
/kaggle/input/ods-churn-24/transactions.csv


In [2]:
import pandas as pd
import numpy as np
# from sksurv.ensemble import RandomSurvivalForest
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import random
import warnings
from IPython.display import display, HTML

# pd.set_option('display.max_columns', None)
warnings.simplefilter('ignore')
# pd.options.display.max_columns = 100
# pd.options.display.max_rows = 100
random.seed(42)
np.random.seed(42)

In [3]:
clients = pd.read_csv('/kaggle/input/ods-churn-24/clients.csv')
report_dates = pd.read_csv('/kaggle/input/ods-churn-24/report_dates.csv', parse_dates=['report_dt'])
transactions = pd.read_csv('/kaggle/input/ods-churn-24/transactions.csv', parse_dates=['transaction_dttm'])
currency_rk = pd.read_csv('/kaggle/input/ods-churn-24/currency_rk.csv')
currency_mult = pd.DataFrame({'currency_rk': [0,1,2,3], 'mult':[24,1,100,90]})
mcc_codes = pd.read_csv('/kaggle/input/ods-churn-24/mcc_codes.csv')
sample_submit_naive = pd.read_csv('/kaggle/input/ods-churn-24/sample_submit_naive.csv')
train = pd.read_csv('/kaggle/input/ods-churn-24/train.csv')

transactions = transactions.merge(currency_mult, how='left')
transactions.transaction_amt = transactions.transaction_amt * transactions.mult
transactions.drop(columns=['mult'], inplace=True)

In [4]:
transactions['min_trans_dt'] = transactions.groupby(['user_id'])['transaction_dttm'].transform('min')
transactions['delta'] = (transactions['transaction_dttm'] - transactions['min_trans_dt']).dt.total_seconds().fillna(0) / (3600*24)
transactions['transaction_amt'] = transactions['delta']
transactions = transactions.drop(['min_trans_dt', 'delta'], axis=1)

#### FeatureTools

In [5]:
import featuretools as ft

# Load the datasets
train_df = clients[['user_id']].copy()
transactions_df = transactions.drop(['currency_rk'], axis=1).copy()

# train_df = clients[['user_id']][0:500].copy()
# transactions_df = transactions[0:5000].drop(['currency_rk'], axis=1).copy()

# Convert transaction_dttm to datetime
transactions_df['transaction_dttm'] = pd.to_datetime(transactions_df['transaction_dttm'])

# Create an EntitySet
es = ft.EntitySet(id='user_transactions')

# Add the transactions dataframe to the EntitySet
es = es.add_dataframe(
    dataframe_name='transactions',
    dataframe=transactions_df,
    index='transaction_id',
    time_index='transaction_dttm',
    logical_types={
        'mcc_code': 'Categorical',
        'transaction_amt': 'Double',  # Corrected logical type
        'transaction_dttm': 'Datetime'
    }
)

# Add the train dataframe to the EntitySet
es = es.add_dataframe(
    dataframe_name='train',
    dataframe=train_df,
    index='user_id'
)

# Correctly defining the relationship
es = es.add_relationships([("train", "user_id", "transactions", "user_id")])

# Set interesting values (example)
# transactions_df['mcc_code'].value_counts().nlargest(10).index.tolist() could be used to find interesting mcc_codes
# Here, you manually define them or use a method to find them dynamically
interesting_mcc_codes = transactions['mcc_code'].value_counts(normalize=True).index.tolist()  # Example MCC codes, replace with relevant codes for your case
es['transactions']['mcc_code'].interesting_values = interesting_mcc_codes

# Automatically generate features with expanded primitives and max_depth
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name='train',
    agg_primitives=[
        'sum', 'mean', 'max', 'min', 'count', 'percent_true', 'num_unique', 'mode', 'skew',
        'trend', 'std'
    ],  # Expanded list of aggregation primitives
    trans_primitives=[
        'day', 'month', 'year', 'weekday', 'is_weekend',
        'time_since_previous', 'cum_sum', 'percentile', 'diff'
    ],  # Expanded list of transformation primitives
    where_primitives=['sum', 'mean', 'max'],  # Conditional aggregations
    max_depth=3,  # Increased max_depth for more complex features
)

In [6]:
main_ft = clients[['user_id']].copy()
main_ft = main_ft.merge(feature_matrix, how='left', on='user_id')
main_ft

,user_id,COUNT(transactions),MAX(transactions.transaction_amt),MEAN(transactions.transaction_amt),MIN(transactions.transaction_amt),MODE(transactions.mcc_code),NUM_UNIQUE(transactions.mcc_code),SKEW(transactions.transaction_amt),STD(transactions.transaction_amt),SUM(transactions.transaction_amt),...,PERCENTILE(STD(transactions.PERCENTILE(transaction_amt))),PERCENTILE(STD(transactions.TIME_SINCE_PREVIOUS(transaction_dttm))),PERCENTILE(SUM(transactions.CUM_SUM(transaction_amt))),PERCENTILE(SUM(transactions.DIFF(transaction_amt))),PERCENTILE(SUM(transactions.PERCENTILE(transaction_amt))),PERCENTILE(SUM(transactions.TIME_SINCE_PREVIOUS(transaction_dttm))),"PERCENTILE(TREND(transactions.CUM_SUM(transaction_amt), transaction_dttm))","PERCENTILE(TREND(transactions.DIFF(transaction_amt), transaction_dttm))","PERCENTILE(TREND(transactions.PERCENTILE(transaction_amt), transaction_dttm))","PERCENTILE(TREND(transactions.TIME_SINCE_PREVIOUS(transaction_dttm), transaction_dttm))"
0,3,11,106.357083,59.161920,0.0,0,4,-0.334024,40.687321,650.781123,...,0.171510,0.075479,0.022323,0.475948,0.014292,0.033969,0.264229,0.035542,0.908823,0.351208
1,9,90,181.297581,84.936935,0.0,1,22,-0.075215,52.556077,7644.324109,...,0.644000,0.833292,0.092646,0.870073,0.535125,0.721422,0.015979,0.167813,0.546875,0.073656
2,13,22,168.358495,80.059439,0.0,3,4,0.189786,51.496458,1761.307662,...,0.566906,0.857802,0.176271,0.422312,0.159188,0.281443,0.714125,0.585010,0.529958,0.143000
3,37,315,179.022535,96.490603,0.0,2,28,-0.255623,46.110445,30394.539850,...,0.275781,0.627594,0.813198,0.858854,0.907792,0.909474,0.583927,0.667854,0.311344,0.197500
4,41,16,153.207130,85.719540,0.0,3,5,-0.199816,43.762566,1371.512639,...,0.218312,0.814073,0.010594,0.595458,0.104667,0.202208,0.081542,0.060750,0.681427,0.038458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95995,562043,37,123.724942,41.421159,0.0,1,12,1.729192,19.736875,1532.582882,...,0.012344,0.989729,0.523594,0.151250,0.139260,0.427542,0.339531,0.925396,0.891156,0.980010
95996,562205,151,178.350035,92.091572,0.0,11,15,-0.120432,49.396821,13905.827384,...,0.416167,0.812937,0.871229,0.080927,0.703146,0.792807,0.253667,0.149490,0.291948,0.903667
95997,562312,56,157.983009,70.069111,0.0,1,7,0.454147,49.581105,3923.870220,...,0.448177,0.862521,0.640906,0.227781,0.357833,0.541943,0.292198,0.150979,0.625208,0.879896
95998,562721,85,174.884363,69.709890,0.0,3,11,0.586322,42.024891,5925.340683,...,0.174760,0.969094,0.741865,0.113281,0.469500,0.871708,0.297885,0.268135,0.555271,0.853625


#### TSFresh

In [7]:
from tsfresh import extract_features
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.utilities.dataframe_functions import impute

# Load the transaction data
transactions_df = transactions.drop(['currency_rk'], axis=1).copy()

# transactions_df = transactions[0:5000].drop(['currency_rk'], axis=1).copy()

# Ensure 'transaction_dttm' is a datetime type, assuming it's already in an appropriate datetime format
transactions_df['transaction_dttm'] = pd.to_datetime(transactions_df['transaction_dttm'])

# Setting up the data in a format suitable for TSFresh
# Assuming 'user_id' is in your transactions_df to identify different users
# We will use 'transaction_dttm' as the time index
transactions_df_sorted = transactions_df.sort_values(by=['user_id', 'transaction_dttm'])

# Extract features
extraction_settings = ComprehensiveFCParameters()

# The 'column_id' is assumed to be 'user_id' to treat each user's data as a separate time series
# The 'column_sort' is 'transaction_dttm' to sort transactions in time order
# 'column_value' could be 'transaction_amt' if you're interested in extracting features from the transaction amounts
extracted_features = extract_features(
    transactions_df_sorted,
    column_id='user_id',
    column_sort='transaction_dttm',
    column_value='transaction_amt',  # Or any other column you want to analyze
    default_fc_parameters=extraction_settings,
    impute_function=impute  # Impute missing values generated during feature extraction
)

Feature Extraction: 100%|██████████| 10/10 [2:48:47<00:00, 1012.77s/it]


In [8]:
extracted_features['user_id'] = extracted_features.index

main_tsf = clients[['user_id']].copy()
main_tsf = main_tsf.merge(extracted_features, how='left', on='user_id')
main_tsf

,user_id,transaction_amt__variance_larger_than_standard_deviation,transaction_amt__has_duplicate_max,transaction_amt__has_duplicate_min,transaction_amt__has_duplicate,transaction_amt__sum_values,transaction_amt__abs_energy,transaction_amt__mean_abs_change,transaction_amt__mean_change,transaction_amt__mean_second_derivative_central,...,transaction_amt__fourier_entropy__bins_5,transaction_amt__fourier_entropy__bins_10,transaction_amt__fourier_entropy__bins_100,transaction_amt__permutation_entropy__dimension_3__tau_1,transaction_amt__permutation_entropy__dimension_4__tau_1,transaction_amt__permutation_entropy__dimension_5__tau_1,transaction_amt__permutation_entropy__dimension_6__tau_1,transaction_amt__permutation_entropy__dimension_7__tau_1,transaction_amt__query_similarity_count__query_None__threshold_0.0,transaction_amt__mean_n_absolute_max__number_of_maxima_7
0,3,1.0,0.0,0.0,0.0,650.781123,5.505604e+04,10.635708,10.635708,-0.000838,...,0.450561,0.867563,1.791759,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,84.472674
1,9,1.0,0.0,0.0,0.0,7644.324109,8.951160e+05,2.037051,2.037051,-0.010306,...,0.104732,0.104732,0.282600,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,168.669191
2,13,1.0,0.0,0.0,0.0,1761.307662,1.966989e+05,8.017071,8.017071,-0.182066,...,0.286836,0.286836,0.836988,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,141.985098
3,37,1.0,0.0,0.0,0.0,30394.539850,3.600406e+06,0.570135,0.570135,0.000793,...,0.045395,0.045395,0.136002,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,171.205904
4,41,1.0,0.0,0.0,0.0,1371.512639,1.462929e+05,10.213809,10.213809,-1.030696,...,0.348832,0.683739,1.889159,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,125.427755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95995,562043,1.0,0.0,0.0,0.0,1532.582882,7.750495e+04,3.436804,3.436804,0.747857,...,0.206192,0.206192,0.926908,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,68.807545
95996,562205,1.0,0.0,0.0,0.0,13905.827384,1.646616e+06,1.189000,1.189000,-0.001931,...,0.070054,0.070054,0.139933,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,175.413523
95997,562312,1.0,0.0,0.0,0.0,3923.870220,4.101478e+05,2.872418,2.872418,0.020794,...,0.149995,0.149995,0.689379,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,151.734567
95998,562721,1.0,0.0,0.0,0.0,5925.340683,5.614065e+05,2.081957,2.081957,-0.020723,...,0.110453,0.110453,0.329684,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,151.119241


#### Selection

In [9]:
data = main_ft.merge(train[['user_id', 'target']], how='left')
data = data[~data.target.isna()]
labels = data['target']
data = data.drop(columns=['target', 'user_id'])

print(main_ft.shape)

(96000, 174)


In [10]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold

# Load the datasets
features_df = data.copy()
target_df = labels.copy()

# Assuming 'user_id' is the common key and both datasets are aligned
# If not, you might need to merge or align them based on your specific needs

# Step 1: Remove features with a big share of Missing Values
# Let's assume "a big share" means more than 60% missing
threshold = 0.6  # 60%
features_df = features_df.loc[:, features_df.isnull().mean() < threshold]

# Step 2: Remove features with Single Unique Value
features_df = features_df.loc[:, features_df.apply(pd.Series.nunique) != 1]

# Step 3: Remove duplicate features
# Transpose the dataframe, drop duplicate rows (now columns), and transpose back
features_df = features_df.T.drop_duplicates().T

# Step 4: Remove collinear features
# Calculate the correlation matrix and remove one of two features with correlation higher than a threshold
corr_matrix = features_df.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
features_df = features_df.drop(columns=to_drop)

# If necessary, align the target with the features based on 'user_id' or another identifier
# This step is skipped here but can be performed as needed

# Step 5: Optionally, remove features with low variance
# This requires fitting the data, so ensure to handle missing values if any remain
# Fill missing values with 0, mean, median, or any other method as appropriate for your dataset
features_df.fillna(0, inplace=True)  # Example: filling with 0
selector = VarianceThreshold(threshold=0.01)  # Adjust threshold as needed
features_reduced = selector.fit_transform(features_df)

# Convert back to DataFrame if necessary, for further use
features_final = pd.DataFrame(features_reduced, columns=features_df.columns[selector.get_support()])
good_cols = features_final.columns

main_ft = main_ft[['user_id']+list(good_cols)]
print(main_ft.shape)

(96000, 86)


In [11]:
data = main_tsf.merge(train[['user_id', 'target']], how='left')
data = data[~data.target.isna()]
labels = data['target']
data = data.drop(columns=['target', 'user_id'])

print(main_tsf.shape)

(96000, 784)


In [12]:
# Load the datasets
features_df = data.copy()
target_df = labels.copy()

# Assuming 'user_id' is the common key and both datasets are aligned
# If not, you might need to merge or align them based on your specific needs

# Step 1: Remove features with a big share of Missing Values
# Let's assume "a big share" means more than 60% missing
threshold = 0.6  # 60%
features_df = features_df.loc[:, features_df.isnull().mean() < threshold]

# Step 2: Remove features with Single Unique Value
features_df = features_df.loc[:, features_df.apply(pd.Series.nunique) != 1]

# Step 3: Remove duplicate features
# Transpose the dataframe, drop duplicate rows (now columns), and transpose back
features_df = features_df.T.drop_duplicates().T

# Step 4: Remove collinear features
# Calculate the correlation matrix and remove one of two features with correlation higher than a threshold
corr_matrix = features_df.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
features_df = features_df.drop(columns=to_drop)

# If necessary, align the target with the features based on 'user_id' or another identifier
# This step is skipped here but can be performed as needed

# Step 5: Optionally, remove features with low variance
# This requires fitting the data, so ensure to handle missing values if any remain
# Fill missing values with 0, mean, median, or any other method as appropriate for your dataset
features_df.fillna(0, inplace=True)  # Example: filling with 0
selector = VarianceThreshold(threshold=0.01)  # Adjust threshold as needed
features_reduced = selector.fit_transform(features_df)

# Convert back to DataFrame if necessary, for further use
features_final = pd.DataFrame(features_reduced, columns=features_df.columns[selector.get_support()])
good_cols = features_final.columns

main_tsf = main_tsf[['user_id']+list(good_cols)]
print(main_tsf.shape)

(96000, 353)


#### Importance

In [13]:
train = pd.read_csv('/kaggle/input/ods-churn-24/train.csv')
train = main_ft.merge(train[['user_id', 'target']], how='left')
train = train[~train.target.isna()]
train = train.sort_values('user_id').reset_index(drop=True)

cat_cols = main_ft.select_dtypes(exclude=["number","bool_","object_"]).columns.to_list()

# Обучение модельки для того чтобы получить важные фичи

model = CatBoostClassifier(
    iterations = 1400,
    depth=5,
    learning_rate=0.03,
    
    eval_metric='AUC',
    cat_features = cat_cols,
    thread_count=6,
    early_stopping_rounds=200,
)
model.fit(train.drop(['user_id', 'target'], axis=1), train['target'], verbose=100)


df_imp = pd.DataFrame({
    'name': train.drop(['user_id', 'target'], axis=1).columns,
    'imp': model.get_feature_importance()
}).sort_values('imp', ascending=False)

df_imp = df_imp[df_imp['imp'] > 0.15] # Берем все фичи, у которых важность больше 0.3
df_imp_050 = df_imp[df_imp['imp'] > 0.50]
df_imp_055 = df_imp[df_imp['imp'] > 0.55]
df_imp_060 = df_imp[df_imp['imp'] > 0.60]
df_imp_065 = df_imp[df_imp['imp'] > 0.65]

good_cols = df_imp['name'].tolist()
good_cols_050 = df_imp_050['name'].tolist()
good_cols_055 = df_imp_055['name'].tolist()
good_cols_060 = df_imp_060['name'].tolist()
good_cols_065 = df_imp_065['name'].tolist()

pred = model.predict_proba(train.drop(['user_id', 'target'], axis=1))[:, 1]
print(metrics.roc_auc_score(train['target'], pred))

TBB Warning: The number of workers is currently limited to 3. The request for 5 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.



0:	total: 141ms	remaining: 3m 17s
100:	total: 7.99s	remaining: 1m 42s
200:	total: 16s	remaining: 1m 35s
300:	total: 23.9s	remaining: 1m 27s
400:	total: 31.3s	remaining: 1m 18s
500:	total: 38.8s	remaining: 1m 9s
600:	total: 46.4s	remaining: 1m 1s
700:	total: 54.4s	remaining: 54.3s
800:	total: 1m 1s	remaining: 46.3s
900:	total: 1m 9s	remaining: 38.5s
1000:	total: 1m 17s	remaining: 30.8s
1100:	total: 1m 25s	remaining: 23.2s
1200:	total: 1m 32s	remaining: 15.4s
1300:	total: 1m 40s	remaining: 7.65s
1399:	total: 1m 48s	remaining: 0us
0.8164088130817716


In [14]:
df_imp[0:50]

,name,imp
32,CUM_SUM(COUNT(transactions)),10.438143
46,CUM_SUM(MIN(transactions.TIME_SINCE_PREVIOUS(t...,7.255760
7,MAX(transactions.CUM_SUM(transaction_amt)),4.722275
42,PERCENTILE(NUM_UNIQUE(transactions.mcc_code)),4.347097
4,NUM_UNIQUE(transactions.mcc_code),4.040031
47,CUM_SUM(SUM(transactions.DIFF(transaction_amt))),2.816144
44,PERCENTILE(STD(transactions.transaction_amt)),2.341432
83,PERCENTILE(TREND(transactions.PERCENTILE(trans...,1.999108
40,PERCENTILE(MAX(transactions.transaction_amt)),1.880481
39,PERCENTILE(COUNT(transactions)),1.811542


In [15]:
main_ft = main_ft[['user_id']+good_cols]
main_ft_050 = main_ft[['user_id']+good_cols_050]
main_ft_055 = main_ft[['user_id']+good_cols_055]
main_ft_060 = main_ft[['user_id']+good_cols_060]
main_ft_065 = main_ft[['user_id']+good_cols_065]

In [16]:
train = pd.read_csv('/kaggle/input/ods-churn-24/train.csv')
train = main_tsf.merge(train[['user_id', 'target']], how='left')
train = train[~train.target.isna()]
train = train.sort_values('user_id').reset_index(drop=True)

cat_cols = main_tsf.select_dtypes(exclude=["number","bool_","object_"]).columns.to_list()

# Обучение модельки для того чтобы получить важные фичи

model = CatBoostClassifier(
    iterations = 1400,
    depth=5,
    learning_rate=0.03,
    
    eval_metric='AUC',
#     cat_features = cat_cols,
    thread_count=6,
    early_stopping_rounds=200,
)
model.fit(train.drop(['user_id', 'target'], axis=1), train['target'], verbose=100)


df_imp = pd.DataFrame({
    'name': train.drop(['user_id', 'target'], axis=1).columns,
    'imp': model.get_feature_importance()
}).sort_values('imp', ascending=False)

df_imp = df_imp[df_imp['imp'] > 0.15] # Берем все фичи, у которых важность больше 0.3
df_imp_050 = df_imp[df_imp['imp'] > 0.50]
df_imp_055 = df_imp[df_imp['imp'] > 0.55]
df_imp_060 = df_imp[df_imp['imp'] > 0.60]
df_imp_065 = df_imp[df_imp['imp'] > 0.65]

good_cols = df_imp['name'].tolist()
good_cols_050 = df_imp_050['name'].tolist()
good_cols_055 = df_imp_055['name'].tolist()
good_cols_060 = df_imp_060['name'].tolist()
good_cols_065 = df_imp_065['name'].tolist()

pred = model.predict_proba(train.drop(['user_id', 'target'], axis=1))[:, 1]
print(metrics.roc_auc_score(train['target'], pred))

0:	total: 101ms	remaining: 2m 21s
100:	total: 9s	remaining: 1m 55s
200:	total: 17.2s	remaining: 1m 42s
300:	total: 25.3s	remaining: 1m 32s
400:	total: 33.4s	remaining: 1m 23s
500:	total: 41.6s	remaining: 1m 14s
600:	total: 49.5s	remaining: 1m 5s
700:	total: 57.4s	remaining: 57.3s
800:	total: 1m 5s	remaining: 49.1s
900:	total: 1m 13s	remaining: 40.9s
1000:	total: 1m 21s	remaining: 32.6s
1100:	total: 1m 29s	remaining: 24.3s
1200:	total: 1m 37s	remaining: 16.2s
1300:	total: 1m 45s	remaining: 8.04s
1399:	total: 1m 53s	remaining: 0us
0.8026583137066629


In [17]:
df_imp[0:50]

,name,imp
10,transaction_amt__absolute_sum_of_changes,3.581629
33,transaction_amt__quantile__q_0.9,1.454720
351,transaction_amt__mean_n_absolute_max__number_o...,1.305637
42,transaction_amt__binned_entropy__max_bins_10,1.264583
6,transaction_amt__standard_deviation,1.138400
117,"transaction_amt__fft_coefficient__attr_""real""_...",1.045448
68,transaction_amt__ar_coefficient__coeff_1__k_10,0.984537
16,transaction_amt__cid_ce__normalize_True,0.971748
17,transaction_amt__cid_ce__normalize_False,0.905673
309,transaction_amt__max_langevin_fixed_point__m_3...,0.892005


In [18]:
main_tsf = main_tsf[['user_id']+good_cols]
main_tsf_050 = main_tsf[['user_id']+good_cols_050]
main_tsf_055 = main_tsf[['user_id']+good_cols_055]
main_tsf_060 = main_tsf[['user_id']+good_cols_060]
main_tsf_065 = main_tsf[['user_id']+good_cols_065]

In [19]:
main_ft.to_csv('main_ft.csv', index=False)
main_tsf.to_csv('main_tsf.csv', index=False)

In [20]:
main_ft_050.to_csv('main_ft_050.csv', index=False)
main_ft_055.to_csv('main_ft_055.csv', index=False)
main_ft_060.to_csv('main_ft_060.csv', index=False)
main_ft_065.to_csv('main_ft_065.csv', index=False)
main_tsf_050.to_csv('main_tsf_050.csv', index=False)
main_tsf_055.to_csv('main_tsf_055.csv', index=False)
main_tsf_060.to_csv('main_tsf_060.csv', index=False)
main_tsf_065.to_csv('main_tsf_065.csv', index=False)